In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
import glob
import math
import os

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset
from tensorflow.examples.tutorials.mnist import input_data
tf.logging.set_verbosity(tf.logging.ERROR)
tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
def adjust_image(data):
    # Reshaped to [batch, height, width, channels].
    imgs = tf.reshape(data, [-1, 28, 28, 1])
    # Adjust image size to that in Inception-v3 input.
    imgs = tf.image.resize_images(imgs, (96, 96))
    # Convert to RGB image.
    imgs = tf.image.grayscale_to_rgb(imgs)
    return imgs


def inceptionv3_model_fn(features, labels, mode):
    # Load Inception-v3 model.
    module = hub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_035_96/feature_vector/2")
    input_layer = adjust_image(features["x"])
    outputs = module(input_layer)

    logits = tf.layers.dense(inputs=outputs, units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [ ]:
g = tf.Graph().as_default()
# Load MNIST data.
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
print(train_data)
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
print(eval_labels)
print(train_labels)
# create input functions for train and evaluate methods.
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=10,
    num_epochs=None,
    shuffle=True)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

# Create an estimator
classifier = tf.estimator.Estimator(
    model_fn=inceptionv3_model_fn, model_dir="/tmp/model")
tf.logging.set_verbosity(tf.logging.INFO)
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=10)

# Train network.
classifier.train(
    input_fn=train_input_fn,
    steps=500,
    hooks=[logging_hook])

# Evaluate the model and print results.
eval_results = classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)
g.close()

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
[7 2 1

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_1/depthwise/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_1/depthwise/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_1/depthwise/BatchNorm/gamma:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_1/depthwise/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_1/depthwise/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_1/depthwise/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_1/depthwise/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with Mobile

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_11/depthwise/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_11/depthwise/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_11/depthwise/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_11/depthwise/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_11/depthwise/depthwise_weights:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_11/depthwise/depthwise_weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_11/expand/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/var

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_13/depthwise/depthwise_weights:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_13/depthwise/depthwise_weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_13/expand/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_13/expand/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_13/expand/BatchNorm/gamma:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_13/expand/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_13/expand/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_15/expand/BatchNorm/gamma:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_15/expand/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_15/expand/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_15/expand/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_15/expand/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_15/expand/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_15/expand/weights:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expa

...

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_6/project/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_6/project/BatchNorm/moving_mean
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_6/project/BatchNorm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_6/project/BatchNorm/moving_variance
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_6/project/weights:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_6/project/weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_7/depthwise/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_co

INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_8/project/weights:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_8/project/weights
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/beta:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_9/depthwise/BatchNorm/beta
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/gamma:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_9/depthwise/BatchNorm/gamma
INFO:tensorflow:Initialize variable module/MobilenetV2/expanded_conv_9/depthwise/BatchNorm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/3ab6027928265cfdfe0b88b4e2f3eda811117d45/variables/variables' with MobilenetV2/expanded_conv_9/depthwise/Batc

INFO:tensorflow:probabilities = [[ 0.00114769  0.01000227  0.01430205  0.05544996  0.80529052  0.02096581
   0.00785098  0.05612563  0.00181401  0.02705119]
 [ 0.96620208  0.00030681  0.00278268  0.00227218  0.00123876  0.00195076
   0.00936535  0.0012019   0.00530513  0.00937435]
 [ 0.01368578  0.00282086  0.07591949  0.12841345  0.01861839  0.28187186
   0.2309923   0.01767433  0.07916829  0.15083529]
 [ 0.4270432   0.00045622  0.01880436  0.06654974  0.00397287  0.03110217
   0.1987787   0.00377739  0.21580923  0.0337061 ]
 [ 0.00278522  0.0215225   0.03866375  0.01200394  0.05153456  0.0065262
   0.00390518  0.82822776  0.00172095  0.03310995]
 [ 0.00131669  0.03167634  0.06218605  0.03173205  0.33004373  0.05557382
   0.04155626  0.35424218  0.00573156  0.08594123]
 [ 0.00096516  0.0003684   0.76849103  0.00645393  0.00127454  0.18363571
   0.01961471  0.00952274  0.00190359  0.00777017]
 [ 0.00700806  0.0064056   0.05501384  0.16822967  0.03634456  0.04241368
   0.02441145  0.031

INFO:tensorflow:global_step/sec: 9.03846
INFO:tensorflow:probabilities = [[ 0.00024037  0.96756351  0.00044955  0.00130003  0.0059807   0.0003041
   0.00111344  0.02246838  0.00004769  0.00053239]
 [ 0.00107197  0.00029334  0.09848045  0.58637428  0.00175551  0.10664084
   0.00795547  0.00390241  0.12612188  0.06740393]
 [ 0.8841576   0.00104615  0.00280324  0.00676663  0.00145131  0.00742341
   0.07848436  0.00325846  0.00258193  0.01202686]
 [ 0.00469802  0.1268573   0.03598467  0.01462457  0.53973675  0.01598212
   0.03609718  0.18198915  0.00372483  0.04030538]
 [ 0.96891642  0.00017907  0.00556502  0.00075063  0.00033873  0.00386176
   0.01156149  0.00176935  0.00080387  0.00625358]
 [ 0.00184451  0.00007839  0.00338701  0.09271622  0.00148491  0.00699625
   0.01583033  0.00049323  0.84969842  0.02747073]
 [ 0.01519859  0.00288202  0.07747399  0.05670981  0.00467535  0.00810744
   0.02982487  0.00982389  0.71928704  0.07601698]
 [ 0.00098859  0.00007891  0.01184309  0.64006072  0.

INFO:tensorflow:probabilities = [[ 0.0002641   0.00475752  0.01590942  0.0009554   0.94112915  0.00370663
   0.00082875  0.02504093  0.00070053  0.00670752]
 [ 0.36714631  0.0030005   0.02182332  0.01777298  0.01464544  0.0317059
   0.16899379  0.02905781  0.04297617  0.30287781]
 [ 0.00283465  0.00035588  0.04957155  0.83470225  0.01106533  0.04160837
   0.00930458  0.00399679  0.02593949  0.02062118]
 [ 0.00005161  0.97451979  0.0002762   0.00012186  0.01188626  0.00009423
   0.00028629  0.01147829  0.00018627  0.00109933]
 [ 0.00012997  0.94001192  0.0002929   0.0007434   0.01137367  0.00019784
   0.00065493  0.04353987  0.00046686  0.00258865]
 [ 0.00600907  0.00018064  0.13365705  0.23588043  0.00653447  0.26211345
   0.05285199  0.00171802  0.16893712  0.13211767]
 [ 0.03277089  0.00241235  0.78372467  0.00679922  0.00209783  0.07130989
   0.01625069  0.04866331  0.00149387  0.03447723]
 [ 0.32224002  0.00089338  0.05212583  0.14531523  0.0036489   0.28452945
   0.06446604  0.015

In [ ]:
# if __name__ == "__main__":
#     tf.app.run()